In [ ]:
"""
#6.6 RetrievalQA (18:22)

Stuff : 나의 document 들로 prompt를 stuff(채우기)
Refine : 각각의 document를 지속적으로 업데이트 하면서 정제한다.
Map Reduce : 각각의 document를 입력받아서 각각에 대한 요약 작업을 하고, 요약본을 LLM에게 전잘
Map Rerank : 질문과 관련된 document를 통해 답변 생성하고, 각 점수를 매겨 가장 높은 점수를 획득한 답변과 그 점수를 반환
"""

In [5]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.vectorstores import Chroma,FAISS # 가끔가다 FAISS가 더 좋음
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()
cache_dir = LocalFileStore("./.cache/")

# chunk_size : 얼마나 큰 덩어리로 나눌지 결정할 수 있음
# chunk_overlap : 문장이나 문단을 분할할 때 앞 조각 일부분을 가져옴
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/현진건-운수_좋은_날.txt",encoding="utf-8")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [7]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

chain.run("김첨지의 직업은 무엇인가")

'주어진 새로운 맥락을 고려하여, 김첨지의 직업은 인력거꾼일 수 있습니다. 이는 김첨지가 정거장에서 아낙인을 모셔다 드리겠다는 제안을 하고, 댁이 어디신가요라고 물어보며 여자에게 손을 대는 장면이 나오기 때문입니다. 또한, 김첨지가 전차를 타는 사람을 노리고 있다가 손에 큰 가방을 들고 있는 모습이 등장하는데, 이는 인력거꾼이 짐을 나르는 역할을 수행하는 직업이기 때문입니다.'